# Flow Control and modular arithmetic
<!-- <a href="https://www.bobvila.com/articles/38-the-plumbing-rough-in/" target="_blank"><img src="img/pipes.jpg" width=500px /></a> -->
<a href="https://www.bobvila.com/articles/38-the-plumbing-rough-in/" target="_blank"><img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/pipes.jpg" width=500px /></a>

## PHYS 2600: Scientific Computing

## Lecture 9

 ### Modular arithmetic

Modular arithmetic is concered with how integers behave under **addtion and multiplication modulo** $\mathbf{n}$ for some integer $n$. It's almost the same as regular integer arithmetic, except numbers **wrap‑around** after $n$.

<a href="https://en.wikipedia.org/wiki/File:Clock_group.svg" target="_blank"><img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/Clock_group.png" width=500px /></a>

We divide up all the integers into $n$ sets according to what their remainder is when we divide by $n$. For example, when $n=5$ we have

$$
\begin{aligned}
\lbrace  \dots -10,-5, 0, 5, 10, \dots\rbrace &\sim 0\\
\lbrace  \dots -9,-4, 1, 6, 11, \dots\rbrace &\sim 1\\
\lbrace  \dots -8,-3, 2, 7, 12, \dots\rbrace &\sim 2\\
\lbrace  \dots -7,-2, 3, 8, 13, \dots\rbrace &\sim 3\\
\lbrace  \dots -6,-1, 4, 9, 14, \dots\rbrace &\sim 4\\
\end{aligned}
$$

Since elements in each set are the same as far as mod 5 arithmetic is concerned, we can just pick one representative from each set. The simplest choice is to just use $\mathbb{Z}_5:=\lbrace  0,1,2,3,4 \rbrace$. Everything about arithmetic mod 5 can then be summed up by addition/multiplication tables (sometimes called Cayley tables) for these values.

In [1]:
import numpy as np


def print_table(Zn_op_Zn, op="+"):
    n = len(Zn_op_Zn)
    table = " | ".join([op, *[f"\x1b[43m{_}\x1b[0m" for _ in range(n)]])
    width_table = 4 * n + 1
    table += "\n"
    table += width_table * "-"
    table += "\n"

    for q in range(n):
        q_op_Zn = Zn_op_Zn[q]
        table += " | ".join([f"\x1b[43m{q}\x1b[0m", *[str(_) for _ in q_op_Zn]])
        table += "\n"
        table += width_table * "-"
        table += "\n"
    print(table)


print_table(np.array([[(q1 + q2) % 5 for q1 in range(5)] for q2 in range(5)]), op="+")

+ | 0 | 1 | 2 | 3 | 4
---------------------
0 | 0 | 1 | 2 | 3 | 4
---------------------
1 | 1 | 2 | 3 | 4 | 0
---------------------
2 | 2 | 3 | 4 | 0 | 1
---------------------
3 | 3 | 4 | 0 | 1 | 2
---------------------
4 | 4 | 0 | 1 | 2 | 3
---------------------



The mod 5 additive inverse $q$ is the number that we need to add to $q$ so that we get $0 \pmod 5$:
$$
\begin{aligned}
-0 &\equiv 0 \pmod 5\\
-1 &\equiv 4 \pmod 5\\
-2 &\equiv 3 \pmod 5\\
-3 &\equiv 2 \pmod 5\\
-4 &\equiv 1 \pmod 5\\
\end{aligned}
$$


Multiplication is bit trickier. We're used to thinking of $2^{-1}$ as another notation for $1/2$ or $0.5$, but in mod 5 arithmetic $2^{-1}$ is the **integer** in $\mathbb{Z}_5=\lbrace  0,1,2,3,4 \rbrace$ that we need to multiply $2$ by to get $1 \pmod 5$.

In [2]:
print_table(np.array([[(q1 * q2) % 5 for q1 in range(5)] for q2 in range(5)]), op="*")

* | 0 | 1 | 2 | 3 | 4
---------------------
0 | 0 | 0 | 0 | 0 | 0
---------------------
1 | 0 | 1 | 2 | 3 | 4
---------------------
2 | 0 | 2 | 4 | 1 | 3
---------------------
3 | 0 | 3 | 1 | 4 | 2
---------------------
4 | 0 | 4 | 3 | 2 | 1
---------------------



$$
\begin{aligned}
1^{-1} &\equiv 1 \pmod 5\\
2^{-1} &\equiv 3 \pmod 5\\
3^{-1} &\equiv 2 \pmod 5\\
4^{-1} &\equiv 4 \pmod 5\\
\end{aligned}
$$

 ### Why do we care about any of this?

Large classes of error correction/detection and encryption/decryption algorithms essentially boil down to solving linear algebra problems involving involving modular arithmetic.
 

One more time, let's recall the motivating example of a recipe as an algorithm:

<!-- <img src="img/lec1-meringue.png" width=600px /> -->
<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/lec1-meringue.png" width=600px />

__Flow control__, in the form of __loops__ and __branches__, is key to this recipe!  Boolean logic enables the _conditional tests_ that make flow control possible.

We'll start with branching - from the diagram, looping is just a special case.

## Branching in Python

Branching flow control is provided by the `if` and `else` statements.  They use indented __code blocks__, similar to functions:

In [3]:
x = 0
if x < 0:
    # Branch run if condition is True
    y = -4
    print("x is negative!")
else:
    # Branch run if condition is False
    y = 4
    print("x is positive!")
print("This line is outside the branches, and y is %d." % y)

x is positive!
This line is outside the branches, and y is 4.


Unlike functions, `if`/`else` code blocks _do not_ have their own local scope.  (So we can assign `y` inside a block and it's visible outside.  This could be a problem if we forget to assign `y` in one branch...)

What if we want to branch into more than two possible outcomes?  One way would be __nesting__ a second `if`/`else` statement inside of the outer one, like this:

In [4]:
x = 0
if x < 0:
    print("x is negative!")
else:
    if x > 0:
        print("x is positive!")
    else:
        print("x is zero!")

x is zero!


(Note how the indentation levels are used to denote which code block belongs to what statement.)  This works, but it's a little hard to read due to the position of the `x < 0` and `x > 0` tests.

Python has a third keyword `elif` (short for "else if") which allows us to put both tests on the same level:

In [5]:
x = 1
if x < 0:
    print("x is negative!")
elif x > 0:
    print("x is positive!")
else:
    print("x is zero!")

x is positive!


This is just a cleaner way to write the same code from the previous cell: the second test `x > 0` will still only happen if the _first_ test `x < 0` returns `False`.

These statements (`if`/`elif`/`else`) are called __conditionals__, because which branch the program takes is decided by whether the _condition_ they test is satisfied or not.

The complete rules of syntax for conditionals are: `if` --> __zero or more__ `elif`s --> __optional__ `else`.  This means that a solo `if` block is valid syntax:

In [6]:
x = 0
if x == 0:
    print("x is definitely zero!")
print("Moving on...")

x is definitely zero!
Moving on...


This is equivalent to the following:

In [7]:
if x == 0:
    print("x is definitely zero!")
else:
    pass
print("Moving on...")

x is definitely zero!
Moving on...


The keyword `pass` means "skip this statement and do nothing."  (This might be useful if we have a lot of `elif` cases but we want one of them to do nothing, for example.)

We can make use of "truthiness" in conditionals: any non-Boolean variable in the test will be typecast to Boolean.  This enables code like the following:

In [13]:
s = []

if s:
    print("s is not empty!")
else:
    print("s is empty!")

s is empty!


__Be careful using this:__ I'm mostly telling you about it so you'll understand it in the wild.  Relying too much on truthiness can lead to unclear code and really surprising glitches!  (The safer way to write the above would be `if s == ''`.)

## Loop branching

Remember that one of the key features of computing is __repetition:__.  To enable this, we need to be able to write a __loop__, in which our code "jumps back" some number of statements and then repeats them.  This _must_ include a conditional branch - otherwise we just have an __infinite loop__.

In Python, a loop with a conditional test is created by the `while` keyword:


In [15]:
x = 0
while x < 5:
    print(x)
    x += 2

print("Loop finished! x is:", x)

0
2
4
Loop finished! x is: 6


Notice that we run _until the condition fails_: so even though `print(x)` in the loop only reaches 4, the final value printed on the last line is 6.

Here's a more visual way to compare and contrast the flow control examples we've seen so far: functions, `if`/`else`, and `while`.

<!-- <img src="img/flow-viz.png" width=700px /> -->
<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/flow-viz.png" width=700px />

Notice that the branching in `while` works _exactly_ the same as `if`: the only difference is at the _end_, where `while` jumps back to the top of the block and `if` just keeps going.

I've included calling a function as another example of code flow where the program is _always_ redirected, instead of only conditionally.

What would happen if we had written the test as `x != 5` instead of `x<5`?

In [ ]:
x = 0
while x != 5:
    #    print(x)
    x += 2

print("Loop finished! x is:", x)

This is an example of an __infinite loop__, which is a common way to make your program run forever (usually by accident!)  Notice if you run this cell that the input number is shown as `In [*]:`; the `*` indicates that the cell is still running.  To get out of the loop in Jupyter, we simply stop the kernel with the Kernel menu, `i`,`i` to interrupt, or `0`,`0` to restart.

<!-- <img src="img/while-viz.png" width=400px style="float:right;" /> -->
<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/while-viz.png" width=400px style="float:right;" />

There are a couple more flow control keywords available within any Python loop: `continue` jumps back up to the top of the loop, and "continues" with the next iteration.  `break` "breaks free" of the loop and takes us immediately past the end.

We can also include an `else` block on a `while` loop, and it works just like it would for `if`.  Under normal operations, __there's no reason to do this__, because the `else` block would run every single time our loop finishes!  The only time it makes sense is in combination with `break`, which will skip over any `else` block present, as shown.

We will only rarely encounter these special keywords: they're mostly used for dealing with exceptional cases in our code, and aren't usually part of a normal loop operation.

## Tutorial 9

Time for our next tutorial!  Connect to the server and load `tut09`.  Today we'll start with a worked example, so get ready to follow along with me.
